In [1]:
import json
import glob

import sys, os, shutil
sys.path.append('/home/daniel/research/catkin_ws/src/')
from ip_nempc import ip_nempc

In [2]:
iae_metrics = []
trial_names = []
INPUT_DIR = '/home/daniel/research/catkin_ws/src/hyperparam_optimization/inverted_pendulum/run_logs/fnn_optimization'

for trial in os.listdir(INPUT_DIR):
    if os.path.isdir(INPUT_DIR + '/' + trial):
        trial_dir = INPUT_DIR + '/' + trial
        trial_name = trial[:14]

        try:
            checkpoint_path = glob.glob(trial_dir + '/lightning_logs/version_0/checkpoints/*.ckpt')[0] # get the checkpoint

            with open(trial_dir + '/params.json', 'r') as f:
                config = json.load(f) # get the config file

            print()
            iae_score, control_successful = ip_nempc(checkpoint_path=checkpoint_path, config=config) # run control

            iae_metrics.append(iae_score)
            trial_names.append(trial_name)
            
            if control_successful:
                print(f'Ran control successfully for trial: {trial_name}. IAE={iae_score}')
            else:
                print(f'Control failed for trial: {trial_name}. IAE={iae_score}')
                raise AssertionError
        except:
            shutil.rmtree(trial_dir) # delete the trial directory since there is no checkpoint to use
            continue # no checkpoint, wasn't a good run

min_index = iae_metrics.index(min(iae_metrics))
print()
print(f'Best Trial: {trial_names[min_index]}, with IAE={iae_metrics[min_index]}')
print(f'Total Trials ran: {len(iae_metrics)}')

100%|██████████| 600/600 [00:13<00:00, 43.38it/s]


Ran control successfully for trial: train_e49cea1a. IAE=892.4431200169527



100%|██████████| 600/600 [00:13<00:00, 45.00it/s]

Ran control successfully for trial: train_21f01584. IAE=884.6673453786375

Best Trial: train_21f01584, with IAE=884.6673453786375
Total Trials ran: 2
